<a href="https://colab.research.google.com/github/NishatVasker/University_Projects/blob/master/BCI_EEG_Emotional_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
data_path = '/content/features_raw.csv'
features_raw = pd.read_csv(data_path)

# Display the first few rows of the dataset to understand its structure
features_raw.head(), features_raw.shape


(        Fp1        AF3        F3         F7       FC5       FC1        C3  \
 0  0.057813  -1.335266  4.640480   0.219573  7.473817  2.314842  1.918097   
 1  1.367408  10.259654  3.345409   7.897852 -2.446051 -1.655035 -6.301423   
 2 -1.783132   4.133553 -0.951680  -1.624803 -1.827309 -2.280364 -2.279225   
 3 -3.690217  -0.814000  2.295469   0.901445  8.323679  1.127906  6.356886   
 4  2.137114   6.420466  6.122230  10.015321  3.106394  3.183129  3.658535   
 
           T7       CP5       CP1  ...        Cz        C4         T8  \
 0  -9.257533  9.089943 -7.104519  ... -2.241480  1.415335   2.406646   
 1  -7.290317 -3.546453 -5.705187  ... -2.568397 -5.651418  -0.096730   
 2   9.151344 -0.239575 -0.057604  ... -2.132823 -0.521117   8.605298   
 3  11.642082  9.354154 -1.662478  ... -0.506117 -1.154866  -3.940251   
 4   4.571793  4.917712 -2.325940  ...  1.813907 -6.444635 -27.680880   
 
          CP6       CP2        P4        P8       PO4        O2  Unnamed: 32  
 0  12.8640

In [2]:
# Drop the 'Unnamed: 32' column
features_processed = features_raw.drop(columns=['Unnamed: 32'])

# Check for missing values in the dataset
missing_values_count = features_processed.isnull().sum().sum()

# Normalize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_normalized = scaler.fit_transform(features_processed)

# Check the shape of the processed data to ensure it's correct
processed_data_shape = features_normalized.shape

missing_values_count, processed_data_shape


(0, (8064, 32))

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Simulating a binary classification target variable for demonstration purposes
# In a real scenario, this should be replaced with actual emotion labels
import numpy as np
target = np.random.randint(2, size=features_normalized.shape[0])

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_normalized, target, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
predictions = rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)

accuracy


0.4934903905765654

In [6]:
pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 8.0 MB/s eta 0:00:00


In [8]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import joblib

# Load the dataset
data_path = '/content/features_raw.csv'  # Update this path if using Google Drive
features_raw = pd.read_csv(data_path)

# Drop any unnecessary columns (like 'Unnamed: 32' in your dataset)
features_raw.drop(columns=['Unnamed: 32'], inplace=True)

# Preprocess the data: Here we'll use StandardScaler for normalization
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features_raw)

# For demonstration, let's create a synthetic target variable
# In a real scenario, replace this with actual labels from your dataset
target = np.random.randint(2, size=features_normalized.shape[0])  # Binary classification

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_normalized, target, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
predictions = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(classification_report(y_test, predictions))

# Save the model for later use
model_filename = '/content/eeg_emotion_classifier.joblib'
joblib.dump(rf_classifier, model_filename)

print("Model saved successfully!")


Accuracy: 49.41%
              precision    recall  f1-score   support

           0       0.50      0.53      0.51       816
           1       0.49      0.46      0.47       797

    accuracy                           0.49      1613
   macro avg       0.49      0.49      0.49      1613
weighted avg       0.49      0.49      0.49      1613

Model saved successfully!


**Hyperparameter Tuning**

In [9]:
from sklearn.model_selection import GridSearchCV

# Define a set of parameters to test
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the Grid Search with cross-validation
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           n_jobs=-1,
                           verbose=2)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_


Fitting 5 folds for each of 36 candidates, totalling 180 fits


**Model Evaluation with Cross-Validation:**

In [10]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation and then re-train the model on the entire training set
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
print(f"CV Scores: {cv_scores}")
print(f"CV Mean Score: {np.mean(cv_scores)}")
best_model.fit(X_train, y_train)


CV Scores: [0.4879938  0.50465116 0.49767442 0.51007752 0.50697674]
CV Mean Score: 0.5014747296429064


RandomForestClassifier(max_depth=10, random_state=42)

**Handle Class Imbalance:**

In [11]:
# If you have identified a class imbalance, use class weights or resampling methods.
# This code snippet assumes you're using class weights.
class_weights = 'balanced'  # or a dictionary of weights you've calculated
rf_classifier_balanced = RandomForestClassifier(n_estimators=100,
                                                random_state=42,
                                                class_weight=class_weights)
rf_classifier_balanced.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [14]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size

In [19]:
!pip install gradio-blocks

ERROR: Could not find a version that satisfies the requirement gradio-blocks (from versions: none)
ERROR: No matching distribution found for gradio-blocks


In [22]:
import gradio as gr
import joblib
import numpy as np

# Load your trained model
model = joblib.load('/content/eeg_emotion_classifier.joblib')

# Define a prediction function
def predict_emotion(eeg_data):
    # Assuming eeg_data is a flattened array of EEG feature values
    eeg_data = np.array(eeg_data).reshape(1, -1)
    eeg_data_normalized = scaler.transform(eeg_data)  # Make sure to use the same scaler as during training

    # Make a prediction with the model
    prediction = model.predict(eeg_data_normalized)
    emotion = 'Happy' if prediction[0] == 1 else 'Sad'  # Replace this with your actual emotion mapping
    return emotion

# Define the Gradio interface
iface = gr.Interface(
    fn=predict_emotion,
    inputs=[gr.Number(label=f"EEG Data Feature {i+1}") for i in range(32)],  # Assuming 32 features
    outputs=gr.Textbox(label="Predicted Emotion")
)

# Launch the Gradio app
iface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:857: UserWarning: Expected 1 arguments for function <function predict_emotion at 0x7f8a8847b640>, received 32.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:865: UserWarning: Expected maximum 1 arguments for function <function predict_emotion at 0x7f8a8847b640>, received 32.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://282342abc7aa3a3ef6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
